In [1]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import os


In [2]:


train_dir = 'C:/Users/ekaushik/Desktop/Multimodal-MusicRecommender/data/train/'

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical',
    subset='training'  # Use subset='training' for training data
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical',
    subset='validation'  # Use subset='validation' for validation data
)

emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# Define checkpoint to save the best model
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

emotion_model_info = emotion_model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // 64,
    epochs=4,
    validation_data=val_generator,
    validation_steps=val_generator.samples // 64,
    callbacks=[checkpoint]  # Pass the checkpoint callback
)


Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.


C:\Users\ekaushik\AppData\Local\Temp\ipykernel_9936\3573497251.py:54: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/4
263/358 [=====================>........] - ETA: 3:35 - loss: 1.8238 - accuracy: 0.2490

In [ ]:
from keras.models import load_model
best_model = load_model('best_model.h5')